# Event log analysis 

In [296]:
### IMPORT ###
from pathlib import Path
import pandas as pd


### LOCAL IMPORT ###
from config import config_reader
from utilities import df_read_csv_data

In [297]:
### GLOBALS ###
yaml_config = config_reader.config_read_yaml("config.yml", "config")
# print(yaml_config) # debug
log_dir = str(yaml_config["LOG_DIR"])
stats_dir = str(yaml_config["STATS_DIR"])
csv_sep = str(yaml_config["CSV_SEP"])
# INPUT
level_input = "PAGE" # [PAGE, PARA]
log_file = "edu_event_log_LEVEL_raw_filtered.csv"

# FUNCTIONS

In [298]:
def calculate_session_count_and_percentage(df:pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the distinct session counts and their percentage based on QuizSessionCount.

    Parameters:
    df (pd.DataFrame): A pandas DataFrame with at least two columns: 
        - 'QuizSessionCount': the number of quiz sessions
        - 'sessionID': unique identifier for each session

    Returns:
    pd.DataFrame: A new DataFrame with three columns:
        - 'QuizSessionCount': distinct values of QuizSessionCount
        - 'SessionCount': count of distinct sessionIDs for each QuizSessionCount
        - 'SessionPerc': percentage of SessionCount with respect to the total, rounded to 2 decimal places
    """

    # Count distinct sessionID for each QuizSessionCount
    session_counts = df.groupby('QuizSessionCount')['sessionID'].nunique().reset_index(name='SessionCount')
    
    # Calculate the percentage of session counts relative to the total
    total_sessions = session_counts['SessionCount'].sum()
    session_counts['SessionPerc'] = (session_counts['SessionCount'] / total_sessions * 100).round(2)
    
    return session_counts

In [299]:
def calculate_column_statistics(df: pd.DataFrame, id_column: str, calc_column: str) -> pd.DataFrame:
    """
    Calculate statistics on a specific column, considering only distinct IDs, and return the results 
    without grouping by ID.

    Parameters:
    df (pd.DataFrame): The input pandas DataFrame.
    id_column (str): The name of the column representing the unique identifiers (IDs).
    calc_column (str): The name of the column on which to perform the calculations.

    Returns:
    pd.DataFrame: A new DataFrame with the following columns:
        - '{calc_column}_not_na': count of non-empty values in the calc_column
        - '{calc_column}_min': the minimum value in the calc_column
        - '{calc_column}_max': the maximum value in the calc_column
        - '{calc_column}_avg': the average value in the calc_column
        - '{calc_column}_med': the median value in the calc_column
    """
    
    # Remove duplicate IDs
    df_no_duplicates = df.drop_duplicates(subset=[id_column])
    
    # Filter out rows where the calculation column is NaN
    non_empty_df = df_no_duplicates[df_no_duplicates[calc_column].notna()]
    
    # Calculate the statistics for the calc_column
    col_not_na = non_empty_df[calc_column].count()  # Count of non-NaN values
    col_min = non_empty_df[calc_column].min()       # Minimum value
    col_max = non_empty_df[calc_column].max()       # Maximum value
    col_avg = non_empty_df[calc_column].mean().round(2)      # Mean value
    col_med = non_empty_df[calc_column].median()    # Median value
    
    # Create a DataFrame with the results, using calc_column as prefix
    stats_df = pd.DataFrame({
        f'{calc_column}_not_na': [col_not_na],
        f'{calc_column}_min': [col_min],
        f'{calc_column}_max': [col_max],
        f'{calc_column}_avg': [col_avg],
        f'{calc_column}_med': [col_med]
    })
    
    return stats_df

In [300]:
def label_terciles_by_session(df: pd.DataFrame, session_column: str, value_column: str):
    """
    Label rows in terciles based on the value_column, considering all rows with the same session_column value 
    as belonging to the same tercile, and add the 'Tercile' column to the original DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    session_column (str): The column representing session IDs (grouping key).
    value_column (str): The column containing the values to be split into terciles.
    
    Returns:
    pd.DataFrame: The original DataFrame with an additional column 'Tercile' indicating the tercile label.
    string: Name of the new tercile column
    """

    # Define the tercile column name
    col_tercile = f"{value_column}_Tercile"

    # First, remove duplicates based on session_column and value_column, because same session has same SUS
    df_unique = df.drop_duplicates(subset=[session_column, value_column])
    
    # Filter out rows where the value_column is NaN
    df_unique_non_nan = df_unique[df_unique[value_column].notna()]
    
    # Calculate terciles based on the unique non-NaN values
    df_unique_non_nan[col_tercile] = pd.qcut(df_unique_non_nan[value_column], q=3, labels=[1, 2, 3], duplicates='drop')
    
    # Merge the tercile labels back into the original dataframe
    df = df.merge(df_unique_non_nan[[session_column, col_tercile]], on=session_column, how='left')
    
    return df, col_tercile

# MAIN

In [301]:
print(">> Setings")
log_file_name = log_file.replace("LEVEL", level_input)
print("Input file:", log_file_name)
path_log_file = Path(log_dir) /log_file_name 
print("Path file:", path_log_file)

>> Setings
Input file: edu_event_log_PAGE_raw_filtered.csv
Path file: data_log/edu_event_log_PAGE_raw_filtered.csv


In [302]:
print(">> Reading")
df_log = df_read_csv_data(path_log_file, None, csv_sep)

>> Reading
Data preview
                                           sessionID pageTitle    menu  \
0  keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...     INTRO  menu_1   
1  keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...   INTRO-Q  menu_1   
2  keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...      PROG  menu_1   
3  keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...    PROG-Q  menu_1   
4  keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...      VARS  menu_1   

   pageOrder  pagePara eventPage       eventTimestamp  click_num  dbclick_num  \
0          1         0    PageIN  2023-06-19 16:25:59          0            0   
1          1         0    PageIN  2023-06-19 16:26:02          0            0   
2          2         0    PageIN  2023-06-19 16:26:09          0            0   
3          2         0    PageIN  2023-06-19 16:26:11          0            0   
4          3         0    PageIN  2023-06-19 16:26:14          0            0   

   QuizSessionCount  ...  Q_28  SUS  Apprend

In [303]:
print("Distinct cases:", df_log["sessionID"].nunique())

Distinct cases: 909


In [304]:
df_log.head(5)

,sessionID,pageTitle,menu,pageOrder,pagePara,eventPage,eventTimestamp,click_num,dbclick_num,QuizSessionCount,...,Q_28,SUS,Apprendimento percepito,UEQ - Pragmatic,UEQ - Hedonic,UEQ - Overall,TotalTimeHH,TotalTimeDD,CaseLength,Class
0,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,INTRO,menu_1,1,0,PageIN,2023-06-19 16:25:59,0,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6,NaN
1,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,INTRO-Q,menu_1,1,0,PageIN,2023-06-19 16:26:02,0,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6,NaN
2,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,PROG,menu_1,2,0,PageIN,2023-06-19 16:26:09,0,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6,NaN
3,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,PROG-Q,menu_1,2,0,PageIN,2023-06-19 16:26:11,0,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6,NaN
4,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,VARS,menu_1,3,0,PageIN,2023-06-19 16:26:14,0,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6,NaN


In [305]:
df_log.columns

Index(['sessionID', 'pageTitle', 'menu', 'pageOrder', 'pagePara', 'eventPage',
       'eventTimestamp', 'click_num', 'dbclick_num', 'QuizSessionCount',
       'QuizAnswerCorrectTotal', 'QuizAnswerWrongTotal',
       'QuizAnswerCorrectRatio', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'Q_5', 'Q_6',
       'Q_7', 'Q_8', 'Q_9', 'Q_10', 'Q_11', 'Q_12', 'Q_13', 'Q_14', 'Q_15',
       'Q_16', 'Q_17', 'Q_18', 'Q_19', 'Q_20', 'Q_21', 'Q_22', 'Q_23', 'Q_24',
       'Q_25', 'Q_26', 'Q_27', 'Q_28', 'SUS', 'Apprendimento percepito',
       'UEQ - Pragmatic', 'UEQ - Hedonic', 'UEQ - Overall', 'TotalTimeHH',
       'TotalTimeDD', 'CaseLength', 'Class'],
      dtype='object')

In [306]:
df_log.shape

(18899, 50)

## Classes

In [307]:
df_log["Class"].unique()

array([nan, 'SAA', 'ECO', 'SMCN1', 'SMTO2', 'SMCN2', 'SMTO1', 'SMTO3'],
      dtype=object)

## Quizzes

In [308]:
print(">> Checking quiz sessions")
df_log["QuizSessionCount"].unique()

>> Checking quiz sessions


array([ 3,  0,  1,  2,  4, 10,  5,  7,  9,  6,  8, 11])

In [309]:
result_df = calculate_session_count_and_percentage(df_log)

In [310]:
result_df

,QuizSessionCount,SessionCount,SessionPerc
0,0,25,2.75
1,1,4,0.44
2,2,16,1.76
3,3,63,6.93
4,4,50,5.50
5,5,50,5.50
6,6,28,3.08
7,7,32,3.52
8,8,29,3.19
9,9,57,6.27


In [311]:
print("> Saving quiz counts")
path_out = Path(stats_dir) / "quiz_count.csv"
print("File out:", path_out)
result_df.to_csv(path_out, index=False, sep=";")

> Saving quiz counts
File out: stats/quiz_count.csv


## Stats on experience

In [312]:
print(">> Stats on experiences")
list_col = ["SUS", "Apprendimento percepito", "UEQ - Pragmatic", "UEQ - Hedonic", "UEQ - Overall"]
print("Columns:", list_col)

>> Stats on experiences
Columns: ['SUS', 'Apprendimento percepito', 'UEQ - Pragmatic', 'UEQ - Hedonic', 'UEQ - Overall']


In [313]:
for col_name in list_col:
    print("Stats on column:", col_name)
    exp_df = calculate_column_statistics(df_log, "sessionID", col_name)
    file_put = f"{col_name}_stats.csv"
    path_out = Path(stats_dir) / file_put
    print("Saving stats to:", path_out)
    exp_df.to_csv(path_out, index=False, sep=csv_sep)
    print()

Stats on column: SUS
Saving stats to: stats/SUS_stats.csv

Stats on column: Apprendimento percepito
Saving stats to: stats/Apprendimento percepito_stats.csv

Stats on column: UEQ - Pragmatic
Saving stats to: stats/UEQ - Pragmatic_stats.csv

Stats on column: UEQ - Hedonic
Saving stats to: stats/UEQ - Hedonic_stats.csv

Stats on column: UEQ - Overall
Saving stats to: stats/UEQ - Overall_stats.csv



## Terciles

In [314]:
print(">> Terciles")
list_col = ["SUS", "Apprendimento percepito", "UEQ - Overall"] # Columns on which to calculate the tertile
print("Columns:", list_col)

>> Terciles
Columns: ['SUS', 'Apprendimento percepito', 'UEQ - Overall']


In [315]:
for col_name in list_col:
    print("Tercile on column:", col_name)
    df_log, col_tercile = label_terciles_by_session(df_log, session_column='sessionID', value_column=col_name)
    print("New tercile column:", col_tercile)
    print("Event log shape:", df_log.shape)
    print("Event log new tercile:", df_log[col_tercile].unique())
    print()

Tercile on column: SUS
New tercile column: SUS_Tercile
Event log shape: (18988, 51)
Event log new tercile: [NaN, 1, 3, 2]
Categories (3, int64): [1 < 2 < 3]

Tercile on column: Apprendimento percepito
New tercile column: Apprendimento percepito_Tercile
Event log shape: (19166, 52)
Event log new tercile: [NaN, 1, 2, 3]
Categories (3, int64): [1 < 2 < 3]

Tercile on column: UEQ - Overall
New tercile column: UEQ - Overall_Tercile
Event log shape: (19522, 53)
Event log new tercile: [NaN, 2, 1, 3]
Categories (3, int64): [1 < 2 < 3]



/var/folders/_n/gwd13q2j4zd4f80gls4mbcmw0000gn/T/ipykernel_14722/1594292989.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_non_nan[col_tercile] = pd.qcut(df_unique_non_nan[value_column], q=3, labels=[1, 2, 3], duplicates='drop')
/var/folders/_n/gwd13q2j4zd4f80gls4mbcmw0000gn/T/ipykernel_14722/1594292989.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_non_nan[col_tercile] = pd.qcut(df_unique_non_nan[value_column], q=3, labels=[1, 2, 3], duplicates='drop')
/var/folders/_n/gwd13q

In [316]:
df_log.head(5)

,sessionID,pageTitle,menu,pageOrder,pagePara,eventPage,eventTimestamp,click_num,dbclick_num,QuizSessionCount,...,UEQ - Pragmatic,UEQ - Hedonic,UEQ - Overall,TotalTimeHH,TotalTimeDD,CaseLength,Class,SUS_Tercile,Apprendimento percepito_Tercile,UEQ - Overall_Tercile
0,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,INTRO,menu_1,1,0,PageIN,2023-06-19 16:25:59,0,0,3,...,NaN,NaN,NaN,0.0,0.0,6,NaN,NaN,NaN,NaN
1,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,INTRO-Q,menu_1,1,0,PageIN,2023-06-19 16:26:02,0,0,3,...,NaN,NaN,NaN,0.0,0.0,6,NaN,NaN,NaN,NaN
2,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,PROG,menu_1,2,0,PageIN,2023-06-19 16:26:09,0,0,3,...,NaN,NaN,NaN,0.0,0.0,6,NaN,NaN,NaN,NaN
3,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,PROG-Q,menu_1,2,0,PageIN,2023-06-19 16:26:11,0,0,3,...,NaN,NaN,NaN,0.0,0.0,6,NaN,NaN,NaN,NaN
4,keaojps50dlKstwSXYup2AVdGzQIsjuXRW88YwXfvOYmVb...,VARS,menu_1,3,0,PageIN,2023-06-19 16:26:14,0,0,3,...,NaN,NaN,NaN,0.0,0.0,6,NaN,NaN,NaN,NaN


## Saving updated event log

In [317]:
print(">> Saving updated event log")
file_name = Path(log_file_name).stem
path_out = Path(log_dir) / f"{file_name}_terziles.csv"
print("Saving event log to:", path_out)
df_log.to_csv(path_out, sep=csv_sep, index=False)

>> Saving updated event log
Saving event log to: data_log/edu_event_log_PAGE_raw_filtered_terziles.csv
